# <u>Training Exercices - Time Series: Evaluation and Illustration<u>

__<u>Jorge Pineño Pérez<u>__ / Module 7 - Time Series

In [55]:
# package imports
install.packages("readxl")
install.packages("dplyr")
install.packages("lmtest")
install.packages("dynlm")
install.packages("zoo")
install.packages("egcm")
library(readxl)
library(dplyr)
library(lmtest)
library(dynlm)
library(zoo)
library(egcm)

Warning message:
"package 'readxl' is in use and will not be installed"
Warning message:
"package 'dplyr' is in use and will not be installed"
Warning message:
"package 'lmtest' is in use and will not be installed"
Warning message:
"package 'dynlm' is in use and will not be installed"
Warning message:
"package 'zoo' is in use and will not be installed"
Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

also installing the dependencies 'urca', 'pracma'




package 'urca' successfully unpacked and MD5 sums checked
package 'pracma' successfully unpacked and MD5 sums checked
package 'egcm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpK0xz9f\downloaded_packages


Cargando paquete requerido: xts


######################### Warning from 'xts' package ##########################
#                                                                             #
# The dplyr lag() function breaks how base R's lag() function is supposed to  #
# work, which breaks lag(my_xts). Calls to lag(my_xts) that you type or       #
# source() into this session won't work correctly.                            #
#                                                                             #
# Use stats::lag() to make sure you're not using dplyr::lag(), or you can add #
# conflictRules('dplyr', exclude = 'lag') to your .Rprofile to stop           #
# dplyr from breaking base R's lag() function.                                #
#                                                                             #
# Code in packages is not affected. It's protected by R's namespace mechanism #
# Set `options(xts.warn_dplyr_breaks_lag = FALSE)` to suppress this warning.  #
#     

<u>__Training Exercise 6.4__<u>

In [2]:
# import the RPK dataframe
RPK <- read_excel('TrainExer64.xlsx')
head(RPK)

YEAR,RPK1,RPK2,X1,X2,DX1,DX2
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1975,13.54875,11.09278,2.606294,2.406294,NA,NA
1976,13.41394,11.09278,2.596294,2.406294,-0.01,0.00
1977,13.41394,11.20426,2.596294,2.416294,0.00,0.01
1978,13.68492,11.20426,2.616294,2.416294,0.02,0.00
1979,14.24341,11.20426,2.656294,2.416294,0.04,0.00
1980,14.53115,11.66152,2.676294,2.456294,0.02,0.04


In [52]:
# convert the tibble to a zoo object of time-series for later use
RPK_ts <- zoo(cbind(
    X1 = RPK$X1,
    X2 = RPK$X2,
    DX1 = RPK$DX1,
    DX2 = RPK$DX2    
), order.by=RPK$YEAR)
head(RPK_ts)

           X1       X2   DX1  DX2
1975 2.606294 2.406294    NA   NA
1976 2.596294 2.406294 -0.01 0.00
1977 2.596294 2.416294  0.00 0.01
1978 2.616294 2.416294  0.02 0.00
1979 2.656294 2.416294  0.04 0.00
1980 2.676294 2.456294  0.02 0.04

__(a)__

In [9]:
# perform the Granger (F-)test of "X2" for "X1" including
# a constant, and two lags of both "X2" and "X1", and check the
# degrees of freedom (Df) and the p-value (Pr(>F))
gt1 <- grangertest(DX1 ~ DX2, order=2, data=RPK)
gt1

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,33,NA,NA,NA
2,35,-2,2.149846,0.132539


In [10]:
# perform the Granger (F-)test of "X1" for "X2" including
# a constant, and two lags of both "X2" and "X1", and check the
# degrees of freedom (Df) and the p-value (Pr(>F))
gt2 <- grangertest(DX2 ~ DX1, order=2, data=RPK)
gt2

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,33,NA,NA,NA
2,35,-2,20.9781,1.321766e-06


__(b)__

__(i)__

In [53]:
# perform the ADF for "DX1", and check the coefficient
# of the lagged value (L(X1, 1)), its std. error, and its t and p-values
adf1 <- dynlm(DX1 ~ index(RPK_ts) + L(X1, 1) + L(DX1, 1), data=RPK_ts)
summary(adf_1)


Time series regression with "zoo" data:
Start = 1977, End = 2015

Call:
dynlm(formula = DX1 ~ index(RPK_ts) + L(X1, 1) + L(DX1, 1), data = RPK_ts)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.019123 -0.006006 -0.001421  0.004739  0.022106 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -4.5563228  1.6210130  -2.811  0.00804 ** 
index(RPK_ts)  0.0025452  0.0009043   2.815  0.00796 ** 
L(X1, 1)      -0.1814847  0.0656510  -2.764  0.00904 ** 
L(DX1, 1)      0.7606219  0.1342284   5.667 2.12e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.01036 on 35 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.5137,	Adjusted R-squared:  0.472 
F-statistic: 12.32 on 3 and 35 DF,  p-value: 1.174e-05


__(ii)__

In [54]:
# perform the ADF for "DX2", and check the coefficient
# of the lagged value (L(X2, 1)), its std. error, and its t and p-values
adf2 <- dynlm(DX2 ~ index(RPK_ts) + L(X2, 1) + L(DX2, 1), data=RPK_ts)
summary(adf_2)


Time series regression with "zoo" data:
Start = 1977, End = 2015

Call:
dynlm(formula = DX2 ~ index(RPK_ts) + L(X2, 1) + L(DX2, 1), data = RPK_ts)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.035897 -0.004838 -0.000094  0.004530  0.029455 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)
(Intercept)   -4.010123   2.920671  -1.373    0.178
index(RPK_ts)  0.002214   0.001627   1.361    0.182
L(X2, 1)      -0.150500   0.124649  -1.207    0.235
L(DX2, 1)      0.187349   0.185459   1.010    0.319

Residual standard error: 0.01206 on 35 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.1154,	Adjusted R-squared:  0.03958 
F-statistic: 1.522 on 3 and 35 DF,  p-value: 0.2259


__(c)__

In [59]:
# perform the two-step Engle-Granger test, and check
# the values of the regression of the first step, and
# the t and p-value for the ADF in step two
eg <- egcm(RPK_ts$X1, RPK_ts$X2)
summary(eg)

Y[i] =   0.9192 X[i] +   0.0116 + R[i], R[i] =   0.7735 R[i-1] + eps[i], eps ~ N(0,  0.0152^2)
        (0.0174)        (0.0506)                (0.1259)

R[2015] = -0.0111 (t = -0.588)


Unit Root Tests of Residuals
                                                    Statistic    p-value
  Augmented Dickey Fuller (ADF)                        -3.583    0.02322
  Phillips-Perron (PP)                                -16.696    0.07353
  Pantula, Gonzales-Farias and Fuller (PGFF)            0.610    0.13617
  Elliott, Rothenberg and Stock DF-GLS (ERSD)          -2.909    0.02380
  Johansen's Trace Test (JOT)                         -49.177    0.00010
  Schmidt and Phillips Rho (SPR)                       -5.125    0.64659

Variances
  SD(diff(X))          =   0.014667
  SD(diff(Y))          =   0.012362
  SD(diff(residuals))  =   0.016504
  SD(residuals)        =   0.018795
  SD(innovations)      =   0.015160

Half life       =   2.698445
R[last]         =  -0.011055 (t=-0.59)

__(d)__

In [62]:
# estimate an ECM for "DX1"
ecm1 <- dynlm(DX1 ~ L(DX1, 1) + I(L(X2, 1) - 0.92*L(X1, 1)), data=RPK_ts)
summary(ecm1)


Time series regression with "zoo" data:
Start = 1977, End = 2015

Call:
dynlm(formula = DX1 ~ L(DX1, 1) + I(L(X2, 1) - 0.92 * L(X1, 1)), 
    data = RPK_ts)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0178807 -0.0051442 -0.0001095  0.0051187  0.0193354 

Coefficients:
                               Estimate Std. Error t value Pr(>|t|)    
(Intercept)                   -0.004070   0.003283  -1.240 0.223159    
L(DX1, 1)                      1.021536   0.137997   7.403  9.8e-09 ***
I(L(X2, 1) - 0.92 * L(X1, 1))  0.462566   0.106704   4.335 0.000112 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.009168 on 36 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.6082,	Adjusted R-squared:  0.5864 
F-statistic: 27.94 on 2 and 36 DF,  p-value: 4.741e-08


In [65]:
# estimate an ECM for "DX2"
ecm2 <- dynlm(DX2 ~ L(DX2, 1) + I(L(X2, 1) - 0.92*L(X1, 1)), data=RPK_ts)
summary(ecm2)


Time series regression with "zoo" data:
Start = 1977, End = 2015

Call:
dynlm(formula = DX2 ~ L(DX2, 1) + I(L(X2, 1) - 0.92 * L(X1, 1)), 
    data = RPK_ts)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0148308 -0.0064739 -0.0005778  0.0072316  0.0170204 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                    0.01793    0.00241   7.440 8.77e-09 ***
L(DX2, 1)                      0.05641    0.12556   0.449    0.656    
I(L(X2, 1) - 0.92 * L(X1, 1)) -0.44340    0.07702  -5.757 1.47e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.009021 on 36 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.4913,	Adjusted R-squared:  0.463 
F-statistic: 17.38 on 2 and 36 DF,  p-value: 5.205e-06
